In [27]:
import numpy as np
import sys
from tqdm import tqdm
#from numba import jit,njit

nDimensions = 3
charge = 1
stepLength = 1
nParticles = 2
# step length and square inverse
h = 0.001
h2 = 1000000
nCycles = 1000000
thermalization = 1000

def local_energy(r, alpha):
    r_plus = np.zeros((nParticles, nDimensions))
    r_minus = np.zeros((nParticles, nDimensions))
    r_plus[:] = r
    r_minus[:] = r
    wave_function_minus = 0
    wave_function_plus = 0
    wave_function_current = wave_function(r, alpha)
    
    # Kinetic energy, brute force derivations
    kinetic_energy = 0
    for i in range(nParticles):
        for j in range(nDimensions):
            r_plus[i,j] += h
            r_minus[i,j] -= h
            wave_function_minus = wave_function(r_minus, alpha)
            wave_function_plus = wave_function(r_plus, alpha)
            kinetic_energy -= (wave_function_minus + wave_function_plus - 2 * wave_function_current)
            r_plus[i,j] = r[i,j]
            r_minus[i,j] = r[i,j]
    kinetic_energy = 0.5 * h2 * kinetic_energy / wave_function_current
    
    # Potential energy
    potential_energy = 0
    r_single_particle = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        potential_energy -= charge / np.sqrt(r_single_particle)
    
    # Contribution from electron-electron potential
    r12 = 0
    for i in range(nParticles):
        for j in range(i + 1, nParticles):
            r12 = 0
            for k in range(nDimensions):
                r12 += (r[i,k] - r[j,k]) * (r[i,k] - r[j,k])
            potential_energy += 1 / np.sqrt(r12)
    
    return kinetic_energy + potential_energy

def wave_function(r, alpha):
    argument = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        argument += np.sqrt(r_single_particle)
    return np.exp(-argument * alpha)

def solve_VMC(alpha):
    print(f'Theoreical: {(-2*alpha**2+4*charge*alpha-5/4*alpha)*13.6}')

    rOld = np.zeros((nParticles, nDimensions))
    rNew = np.zeros((nParticles, nDimensions))
    waveFunctionOld = 0
    waveFunctionNew = 0
    energySum = 0
    energySquaredSum = 0
    deltaE = 0

    # initial trial positions
    rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)
    rNew = rOld.copy()
    waveFunctionOld = wave_function(rOld, alpha)

    # loop over Monte Carlo cycles
    accept = 0
    for cycle in tqdm(range(nCycles), file=sys.stdout, desc='VMC Cycles'):
        
        rNew = rOld + stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

        waveFunctionNew = wave_function(rNew, alpha)

        if np.random.rand() <= np.float_power(waveFunctionNew/waveFunctionOld, 2):
            rOld = rNew.copy()
            waveFunctionOld = waveFunctionNew
            deltaE = local_energy(rNew, alpha)
            accept += 1

        # update energies
        if cycle > thermalization:
            energySum += deltaE
            energySquaredSum += deltaE**2
        
        # if np.amax(abs(rOld)) > 100:
        #     print(rOld)
        #     tqdm.write("triggered")
        #     rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)
        #     rNew = rOld + stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

        if cycle % 50000 == 10000:
            num = cycle - thermalization
            energy  = energySum        / num
            energy2 = energySquaredSum / num
            variance = energy2 - energy ** 2
            error = np.sqrt(variance / num)
            accept_rate = accept/(cycle+1)/nParticles * 100

            energy *= 2*13.6
            error *= 2*13.6

            tqdm.write("energy (eV): "         + str(round(energy, 2)) +\
                       " ; error (eV): "         + str(round(error,5)) +\
                       " ; acceptance rate (%) = " + str(round(accept_rate,2)))
            
            if error <= 1E-2:
                break
    return energy

In [28]:
alpha = 1.75
solve_VMC(alpha = 1.75)

# alphas = [round(0.69 + (i-5)*0.01,2) for i in range(11)]
# print(f'alphas    : {alphas}')
# print(f'Theoreical: {[round((-2*alpha**2+4*charge*alpha-5/4*alpha)*(-13.6),2) for alpha in alphas]}')

# energies = list(map(solve_VMC, alphas))
# print(energies)

Theoreical: -17.849999999999998
energy (eV): 16.25 ; error (eV): 0.63095 ; acceptance rate (%) = 24.88
energy (eV): 17.82 ; error (eV): 0.25619 ; acceptance rate (%) = 24.38
energy (eV): 17.79 ; error (eV): 0.18745 ; acceptance rate (%) = 24.37 
energy (eV): 17.81 ; error (eV): 0.153 ; acceptance rate (%) = 24.42   
energy (eV): 17.88 ; error (eV): 0.13808 ; acceptance rate (%) = 24.37 
energy (eV): 18.0 ; error (eV): 0.12336 ; acceptance rate (%) = 24.34  
energy (eV): 18.14 ; error (eV): 0.11377 ; acceptance rate (%) = 24.39 
energy (eV): 17.96 ; error (eV): 0.10539 ; acceptance rate (%) = 24.43 
energy (eV): 17.93 ; error (eV): 0.09826 ; acceptance rate (%) = 24.42 
energy (eV): 17.99 ; error (eV): 0.09204 ; acceptance rate (%) = 24.44 
energy (eV): 17.82 ; error (eV): 0.08708 ; acceptance rate (%) = 24.46 
energy (eV): 17.69 ; error (eV): 0.0831 ; acceptance rate (%) = 24.48  
energy (eV): 17.71 ; error (eV): 0.07938 ; acceptance rate (%) = 24.47 
energy (eV): 17.53 ; error (eV): 0

17.673794806988113